In [1]:
!pip install stellargraph

In [2]:
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score
import stellargraph as sg
import os
import networkx as nx
import numpy as np
import pandas as pd

from stellargraph.data import BiasedRandomWalk
from stellargraph import StellarGraph
from stellargraph import datasets
from IPython.display import display, HTML

%matplotlib inline

In [5]:
graph = nx.read_gml('Desktop\graph_node2vec2.gml')
node_subjects = nx.get_node_attributes(graph,'genre')

In [6]:
counter =0
list_of_ids =[]
for key, value in node_subjects.items():
    if len(value) != 0:
        list_of_ids.append(counter)
    counter +=1
print(len(list_of_ids))

17777


In [7]:
index_list = [*range(0, len(node_subjects), 1)] 
node_subjects= pd.Series(list(node_subjects.values())[0:len(node_subjects)], index = index_list, name= "genre")
print(node_subjects)

0        blues
1        blues
2          pop
3          pop
4         rock
         ...  
46642         
46643         
46644         
46645         
46646         
Name: genre, Length: 46647, dtype: object


In [8]:
print(counter)
matrix=[]
for i in range(counter):
    vecForEach = np.zeros(2)
    if i > len(list_of_ids):
        vecForEach[1] = 1 #represents its a song node
    else:
        vecForEach[0] = 1 #represents its a word node
    matrix.append(vecForEach)

index = [*range(0,counter,1)]
list_of_tuples = list(zip(index, matrix))
df = pd.DataFrame(list_of_tuples,  columns = ['index', 'value'])   
df

46647


,index,value
0,0,"[1.0, 0.0]"
1,1,"[1.0, 0.0]"
2,2,"[1.0, 0.0]"
3,3,"[1.0, 0.0]"
4,4,"[1.0, 0.0]"
...,...,...
46642,46642,"[0.0, 1.0]"
46643,46643,"[0.0, 1.0]"
46644,46644,"[0.0, 1.0]"
46645,46645,"[0.0, 1.0]"


In [9]:
i=0
for u,outer_d in graph.nodes(data=True):
    #print(type(df.iloc[i]['value']))
    outer_d['val'] = df.iloc[i]['value']
    i+=1

In [10]:
G = sg.StellarGraph.from_networkx(graph,node_features="val")
G.info()

'StellarGraph: Undirected multigraph\n Nodes: 46647, Edges: 8590568\n\n Node types:\n  default: [46647]\n    Features: float32 vector, length 2\n    Edge types: default-default->default\n\n Edge types:\n    default-default->default: [8590568]\n        Weights: all 1 (default)\n        Features: none'

In [11]:
G.nodes()[0:len(node_subjects)]

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '46637', '46638', '46639', '46640', '46641', '46642', '46643', '46644',
       '46645', '46646'],
      dtype='object', length=46647)

In [12]:
rw = BiasedRandomWalk(G)

walks = rw.run(
    nodes=list(G.nodes()[0:len(list_of_ids)]),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

Number of random walks: 177770


In [13]:
import json
with open("walks.txt", "w") as f:
  json.dump(walks, f)

In [17]:
from gensim.models import Word2Vec

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, vector_size=128, window=5, min_count=0, sg=1, workers=2, epochs=1)

In [ ]:
# The embedding vectors can be retrieved from model.wv using the node ID as key.
# model.wv["19231"].shape

In [19]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
node_targets = node_subjects[[int(node_id) for node_id in node_ids]]

In [20]:
# Apply t-SNE transformation on node embeddings
tsne = TSNE(n_components=2)
node_embeddings_2d = tsne.fit_transform(node_embeddings[0:len(list_of_ids)])

In [1]:
# draw the points
alpha = 0.7
label_map = {l: i for i, l in enumerate(np.unique(node_targets[0:len(list_of_ids)]))}
node_colours = [label_map[target] for target in node_targets[0:len(list_of_ids)]]

plt.figure(figsize=(10, 8))
plt.scatter(
    node_embeddings_2d[:, 0],
    node_embeddings_2d[:, 1],
    c=node_colours,
    cmap="jet",
    alpha=alpha,
)

NameError: name 'np' is not defined

In [22]:
# X will hold the 128-dimensional input features
X = node_embeddings[0:len(list_of_ids)]
# y holds the corresponding target values
y = np.array(node_targets[0:len(list_of_ids)])

In [23]:
node_targets

46646     
17803     
17786     
17872     
17992     
        ..
43971     
45866     
45982     
46231     
34575     
Name: genre, Length: 46645, dtype: object

In [24]:
node_embeddings

array([[-0.32766342,  0.2941799 , -0.3474949 , ..., -0.82027346,
         0.24436763, -0.09175396],
       [-0.05642795, -0.30560762,  0.10986784, ..., -0.3039986 ,
        -0.17142121, -0.13247998],
       [ 0.00373635, -0.06285318, -0.00417267, ..., -0.00314891,
         0.04443538, -0.02185215],
       ...,
       [-0.02786947, -0.1008638 ,  0.01538149, ..., -0.06193265,
        -0.05278542,  0.01718287],
       [-0.04645026, -0.08724491, -0.01361432, ..., -0.03430731,
        -0.04251448,  0.03825265],
       [-0.01508978, -0.0516769 , -0.00201331, ..., -0.03399877,
        -0.03016051,  0.02054489]], dtype=float32)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=None)

In [26]:
print(
    "Array shapes:\n X_train = {}\n y_train = {}\n X_test = {}\n y_test = {}".format(
        X_train.shape, y_train.shape, X_test.shape, y_test.shape
    )
)

Array shapes:
 X_train = (14221, 128)
 y_train = (14221,)
 X_test = (3556, 128)
 y_test = (3556,)


In [27]:
clf = LogisticRegressionCV(
    Cs=10, cv=10, scoring="accuracy", verbose=False, multi_class="ovr", max_iter=300
)
clf.fit(X_train, y_train)

C:\Users\ivang\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ivang\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

LogisticRegressionCV(cv=10, max_iter=300, multi_class='ovr', scoring='accuracy',
                     verbose=False)

In [28]:
y_pred = clf.predict(X_test)

In [29]:
y_test

array(['reggae', '', 'rock', ..., 'pop', 'blues', 'pop'], dtype=object)

In [30]:
y_pred

array(['reggae', '', 'rock', ..., 'pop', 'blues', 'pop'], dtype=object)

In [31]:
accuracy_score(y_test, y_pred)

0.7277840269966255